## Run Single Side Band (SSB) ptychography on Merlin live streams

If you want to use this with the simulated data source, run something like this in the background:

`libertem-live-mib-sim ~/Data/default.hdr --cached=MEM`

On Linux, `MEMFD` is also supported as a cache. Use `NONE` to deactivate the cache.

* Make sure to adjust the `SCAN_SIZE` below to match the scan of the data source!
* This notebook requires the `bqplot` extra of LiberTEM: `pip install libertem[bqplot]`
* This notebook requires the `ptychography40` package from https://github.com/Ptychography-4-0/ptychography

In [1]:
# set this to the host/port where the merlin data server is listening:
MERLIN_DATA_SOCKET = ('127.0.0.1', 6342)
SCAN_SIZE = (128, 128)

In [2]:
import time
import logging
from contextlib import contextmanager

import numpy as np
import ipywidgets

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
from libertem.corrections.coordinates import flip_y, rotate_deg, identity
from libertem.common.container import MaskContainer
from libertem.viz.bqp import BQLive2DPlot

INFO:empyre:Imported EMPyRe V-0.3.0 GIT-7531a074e8e81c3e02d65fad075edcd2c5408ad7


In [5]:
from libertem_live.api import LiveContext

In [6]:
from ptychography40.reconstruction.ssb import SSB_UDF, generate_masks
from ptychography40.reconstruction.common import wavelength, get_shifted

In [7]:
ctx = LiveContext()

### Camera setup

In [8]:
@contextmanager
def medipix_setup(dataset, udfs):
    print("priming camera for acquisition")
    # TODO: medipix control socket commands go here

    # dataset.source.set('numframes', 1024)
    # dataset.source.set(...)

    # microscope.start_scanning()
    print("running acquisition")
    with dataset.start_acquisition():
        yield
    print("camera teardown")
    # teardown routines go here

In [9]:
ds = ctx.prepare_acquisition(
    'merlin',
    medipix_setup,
    scan_size=SCAN_SIZE,
    host=MERLIN_DATA_SOCKET[0],
    port=MERLIN_DATA_SOCKET[1],
    frames_per_partition=800,
    pool_size=2
)

In [10]:
def make_medipix_setup(data_source):
    @contextmanager
    def medipix_setup(dataset, udfs):
        print("priming camera for acquisition")
        # TODO: medipix control socket commands go here?
        
        # mer.set('numframes', 1024)
        # mer.set(...)
        
        with data_source:
            yield
        print("camera teardown")
    return medipix_setup

### SSB setup

See also https://ptychography-4-0.github.io/ptychography/algorithms/ssb.html for a more complete example!

In [11]:
ds_shape_sig, ds_shape_nav = ds.shape.sig, ds.shape.nav

# Acceleration voltage in keV
U = 300
rec_params = {
    "dtype": np.float32,
    "lamb": wavelength(U),
    "dpix": 12.7e-12,
    "semiconv": 22.1346e-3,  # 2020-05-18
    "semiconv_pix": 31,  # 2020-05-18
    # applied right to left
    "transformation": rotate_deg(88) @ flip_y(),
    "cx": 123,
    "cy": 126,
    "cutoff": 16,  # number of pixels: trotters smaller than this will be removed
}
cutoff_freq = np.float32('inf')

mask_params = {
    # Shape of the reconstructed area
    'reconstruct_shape': tuple(ds.shape.nav),
    # Shape of a detector frame
    'mask_shape': tuple(ds.shape.sig),
    # Use the faster shifting method to generate trotters
    'method': 'shift',
}

In [12]:
%%time
trotters = generate_masks(**rec_params, **mask_params)

Wall time: 8.29 s


In [13]:
mask_container = MaskContainer(
    mask_factories=lambda: trotters, dtype=trotters.dtype, count=trotters.shape[0]
)

In [14]:
ssb_udf = SSB_UDF(**rec_params, mask_container=mask_container)

### SSB on live data

In [15]:
p0 = BQLive2DPlot(ds, ssb_udf, channel="phase")
p1 = BQLive2DPlot(ds, ssb_udf, channel="amplitude")

In [16]:
outputs = []

for p in [p0, p1]:
    # Capture the plots to display them in a grid later
    output = ipywidgets.Output()
    with output:
        p.display()
        # Some plot-specific tweaks for grid display
        if isinstance(p, BQLive2DPlot):
            p.figure.fig_margin={'top': 50, 'bottom': 0, 'left': 25, 'right': 25}
            p.figure.layout.width = '400px'
            p.figure.layout.height = '400px'
        elif isinstance(p, MPLLive2DPlot):
            p.fig.tight_layout()
            p.fig.set_size_inches((3, 3))
            p.fig.canvas.toolbar_position = 'bottom'
    outputs.append(output)

In [17]:
ipywidgets.HBox(outputs)

### Sample output

The plots are not preserved when saving the notebook. They look like this:

![sample plot](ssb-live.png)

In [18]:
ctx.run_udf(dataset=ds, udf=[ssb_udf], plots=[p0, p1])

priming camera for acquisition
running acquisition
camera teardown


({'fourier': <BufferWrapper kind=single dtype=complex64 extra_shape=(128, 128)>,
  'complex': <BufferWrapper kind=single dtype=complex64 extra_shape=(128, 128)>,
  'amplitude': <BufferWrapper kind=single dtype=float32 extra_shape=(128, 128)>,
  'phase': <BufferWrapper kind=single dtype=float32 extra_shape=(128, 128)>},)